In [227]:
import pandas as pd
import numpy as np
import requests

In [228]:
#!pip install geopy

In [229]:
import io
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
from geopy.geocoders import Nominatim
from time import sleep

In [230]:
### API Tools ###
def get_api_token():
    r"""
    Function to retrieve API token. Token must be refreshed every hour.
    Parameters
    ----------
    None
    Returns
    -------
    API token
    """
    # URL to retrieve token
    url = "https://apieco.eco-counter-tools.com/token"
    # API username and password
    username = "uc_berkeley_atc_api"
    password = "6A399=A529f54d2dAY6d"
    # API information
    payload={'f': 'json',
    'username': username,
    'password': password,
    'grant_type' : 'password',
    'client_secret' : 'oqxQcZErq2DMn2MdghyQLT8rWpsa', 
    'client_id' : '3YApOOeY_Ab76YV817TC4dCF9roa',
    'referer': f"ttps://apieco.eco-counter-tools.com/"}
    # get API token
    response = requests.request("POST", url, data=payload)
    # return API token
    return response.json()['access_token']

# Note: currently issue where won't let you grab just 1 site, need at least 2
def get_data(sites_json, token, userType, start_date, end_date, step_len = "hour"):
    r"""
    Function to retrieve data for sites in sites_json.
    Parameters
    ----------
    sites_json : json
        json of sites
    token : str
        API token
    userType : int
        type of user data to retrieve
        {1: "pedestrian", 2: "bicycle", 3: "horse", 4: "car", 5: "bus", 6: "minibus", 
        7: "undefined", 8: "motorcycle", 9: "kayak", 13: "e-scooter", 14: "truck"}
    start_date : str
    end_date : str
    step_len : str
        level of data aggregation, e.g., hour, day, month
    Returns
    -------
    dataframe
        count data for sites specified over specified time period
    """
    headers = {'Authorization': "Bearer {}".format(token)}
    df = pd.DataFrame()
    for item in sites_json: 
        site_df = pd.DataFrame()
        for channel_item in item['channels']:
            # check if userType is the mode we want, and if so get data
            if channel_item['userType'] == userType:
                # set up count data request
                site_id = str(channel_item['id'])
                base_url = "https://apieco.eco-counter-tools.com/api/1.0/data/site/" + \
                site_id + \
                "?begin=" + start_date + \
                "T00:00:00&end=" + end_date + "T23:59:59&step=" + step_len
                # get count data
                auth_response = requests.get(base_url, headers=headers)
                # convert JSON to dataframe form
                channel_df = pd.json_normalize(auth_response.json())
                # add site and channel name
                channel_df['id'] = channel_item['id']
                channel_df['site_name'] = item['name']
                channel_df['channel_name'] = channel_item['name']
                channel_df['latitude'] = channel_item['latitude']
                channel_df['longitude'] = channel_item['longitude']
                channel_df['direction'] = channel_item['sens']
                channel_df['type'] = channel_item['type']
                channel_df['installationDate'] = channel_item['installationDate']
                channel_df['counter'] = channel_item['counter']
                channel_df['interval'] = channel_item['interval']
                # add channel_df to site_df
                site_df = pd.concat([site_df, channel_df])
        df = pd.concat([df, site_df])
    # return all data
    return df

In [231]:
# Generate API token
token = get_api_token()

In [232]:
# List all sites
base_url = "https://apieco.eco-counter-tools.com/api/1.0/site"

headers = {'Authorization': "Bearer {}".format(token)}
eco_sites = requests.get(base_url, headers=headers)

In [233]:
%%time
# Get data for first few sites
df = get_data(sites_json = eco_sites.json(),
                   token = token, 
                    userType = 2, 
                    start_date = "2008-01-01", 
                    end_date = "2008-12-31", 
                    step_len = "hour")

CPU times: total: 2min 32s
Wall time: 14min 33s


In [234]:
df.head()

,date,isoDate,counts,status,id,site_name,channel_name,latitude,longitude,direction,type,installationDate,counter,interval
0,2008-01-01T00:00:00+0000,2008-01-01T00:00:00-0800,None,None,103000671,BART AT SPIRE,BART at Spire- Bikes IN to Spire,37.79116,-122.45944,1,real,2010-12-23T00:00:00-0800,X0412011792,60
1,2008-01-01T01:00:00+0000,2008-01-01T01:00:00-0800,None,None,103000671,BART AT SPIRE,BART at Spire- Bikes IN to Spire,37.79116,-122.45944,1,real,2010-12-23T00:00:00-0800,X0412011792,60
2,2008-01-01T02:00:00+0000,2008-01-01T02:00:00-0800,None,None,103000671,BART AT SPIRE,BART at Spire- Bikes IN to Spire,37.79116,-122.45944,1,real,2010-12-23T00:00:00-0800,X0412011792,60
3,2008-01-01T03:00:00+0000,2008-01-01T03:00:00-0800,None,None,103000671,BART AT SPIRE,BART at Spire- Bikes IN to Spire,37.79116,-122.45944,1,real,2010-12-23T00:00:00-0800,X0412011792,60
4,2008-01-01T04:00:00+0000,2008-01-01T04:00:00-0800,None,None,103000671,BART AT SPIRE,BART at Spire- Bikes IN to Spire,37.79116,-122.45944,1,real,2010-12-23T00:00:00-0800,X0412011792,60


In [ ]:
#https://developers.eco-counter.com/
# sens:Direction detection information. {1: IN direction, 2: OUT direction, 5: No direction distinction}

In [ ]:
#eco_sites.json()[0] 

In [ ]:
# j = 0
# for site in eco_sites.json():
#     if site['latitude'] > 40 and site['longitude'] < -120:
#         print('id:', site['id'])
#         print('site name:', site['name'])
#         print('site domain:', site['domain'])
#         print('site lat:', site['latitude'])
#         print('site long:',site['longitude'])
#         print('userType:',site['userType'])
#         print('direction:',site['sens'])
#         print('type:',site['type'])
#         print('installationDate:',site['installationDate'])
#         print(j)
#     j = j + 1

In [ ]:
city_state=df.drop_duplicates(subset=['id'], keep='first').reset_index()

In [ ]:
# lets bring to city, county with the data frame
# def city_state_country(row):
#     coord = f"{row['latitude']}, {row['longitude']}"
#     location = geolocator.reverse(coord, exactly_one=True)
#     address = location.raw['address']
#     city = address.get('city', '')
#     state = address.get('state', '')
#     country = address.get('country', '')
#     row['city'] = city
#     row['state'] = state
#     row['country'] = country
#     return row

In [ ]:
def city_state_country(row):
    coord = f"{row['latitude']}, {row['longitude']}"
    sleep(1)
    location = geolocator.reverse(coord, exactly_one=True, language='en')
    if not location:
        # if you see many in a row, it's probably Nominatim blocking you.
        # if it's just every once in a while, there were just some bad results. 
        print('Failed with coord: ', coord)
        row['city'], row['state'], row['country2'] = None, None, None
        return row
    address = location.raw['address']
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    row['city'] = city
    row['state'] = state
    row['country2'] = country
    return row

In [ ]:
city_state1 = city_state.apply(city_state_country, axis=1)

In [ ]:
city_state2=city_state1[['id','latitude','longitude','city','state','country2']]
city_state3=pd.merge(df,city_state2, left_on=['id','latitude','longitude'],right_on=['id','latitude','longitude'], how='inner')

In [ ]:
city_state3.shape, df.shape

In [ ]:
city_state2.groupby(['city'])['id'].count()

In [ ]:
#df = df.apply(city_state_country, axis=1)
#df.head()

In [ ]:
# save the data
city_state3.to_csv("D:/Bike_ped Exposure Model Project/Eco_data/eco_2008_hourly.csv")